In [1]:
from argparse import Namespace
import math
import os, sys
from dataclasses import dataclass
from pathlib import Path
from typing import Optional, Union
from datetime import datetime

import aiohttp
import datasets
import torch
from accelerate import Accelerator
from accelerate.logging import get_logger
from datasets import DatasetDict, concatenate_datasets, load_dataset
from huggingface_hub import HfApi
from torch.utils.data.dataloader import DataLoader
from tqdm.auto import tqdm

import transformers
from transformers import (
    SchedulerType,
    Wav2Vec2Config,
    Wav2Vec2FeatureExtractor,
    Wav2Vec2ForPreTraining,
    get_scheduler,
    is_wandb_available,
    set_seed,
)
from transformers.models.wav2vec2.modeling_wav2vec2 import _compute_mask_indices, _sample_negative_indices
from transformers.utils import send_example_telemetry
import numpy as np
from omegaconf import DictConfig, OmegaConf
from transformers.trainer_pt_utils import LengthGroupedSampler
import wandb

logger = get_logger(__name__)

# BASED ON:
# https://github.com/huggingface/transformers/blob/main/examples/pytorch/speech-pretraining/run_wav2vec2_pretraining_no_trainer.py

In [2]:
def load_pretraining_config(config_path: Optional[str] = None) -> DictConfig:
    """Load pretraining config from YAML file or use defaults."""
    
    # Default configuration matching your argparse setup
    default_config = OmegaConf.create({
        "tune_hyperparams": False,
        
        # Dataset arguments
        "base_path": ".", # Set to directory containing datasets etc
        "normalize_features": True,
        "group_by_length": False,
        "dataset_name": "pre_rvi_mdb",
        "experiment_name": "pretrain_local",
        "dataset_config_names": ["clean", "clean"],  # Will be required, set in validation
        "dataset_split_names": ["train", "val"],   # Will be required, set in validation
        "data_column_name": "traces",
        
        # Preprocessing arguments
        "preprocessing_num_workers": 4,
        "max_trace_expansion":5,
        
        # Training arguments
        "batch_size": 16,
        "learning_rate": 5e-5, # OPTUNA
        "weight_decay": 0.01,
        "num_train_epochs": 3,
        "max_train_steps": None, #overrides num_train_epochs
        "gradient_accumulation_steps": 1,
        "gradient_checkpointing": True,
        "lr_scheduler_type": "linear",
        "num_warmup_steps": 100,
        "output_dir": None,
        "seed": 0,
        
        # Optimizer arguments
        "adam_beta1": 0.9,
        "adam_beta2": 0.98,
        "adam_epsilon": 1e-6,
        
        # Logging and saving
        "logging_steps": 10,
        "saving_steps": 100,
        
        # Hub arguments
        "push_to_hub": False,
        "hub_model_id": None,
        "hub_token": None,
        
        # Trace-specific arguments
        "sampling_rate": 200000000,  # 200*10**6
        "max_trace_length": 10000,  # 10**6
        "min_trace_length": 0,
        "pad_to_multiple_of": None,
        
        # Gumbel softmax arguments
        "max_gumbel_temperature": 2.0,
        "min_gumbel_temperature": 0.5,
        "gumbel_temperature_decay": 0.999995,
        
        # Masking arguments
        "mask_time_prob": 0.5,
        "mask_time_length": 4,

        # Model configuration
        "model": {
            # Core architecture parameters
            "hidden_size": 64,
            "num_hidden_layers": 12, # OPTUNA, [6, 8, 12]
            "num_attention_heads": 4, # OPTUNA, [2,4,8]
            
            # Feature extraction parameters
            "conv_dim": [256, 128, 64],
            "conv_stride": [4, 2, 1], # OPTUNA, [[4, 2, 2], [2, 2, 1]]
            "conv_kernel": [8, 4, 3],
            
            # Normalization settings
            "feat_extract_norm": "layer",
            "layer_norm_eps": 1e-5,
            "do_stable_layer_norm": True, # Test without normalization - maybe only the one in the feature extractor?
            
            # Quantization/codebook parameters
            "vocab_size": 16, # FIXME: This is only relevant during finetuning, adapt in finetuning file.
            "num_codevectors_per_group": 8, # OPTUNA, [6, 8, 10]
            "num_codevector_groups": 2, # OPTUNA, [1, 2]
            "num_negatives": 100,
            
            # Regularization
            "hidden_dropout": 0.1,
            "attention_dropout": 0.1,
            "activation_dropout": 0.0,
            "feat_proj_dropout": 0.0,
            "layerdrop": 0.05,
            "diversity_loss_weight": 1,
            
            # Activation function
            "hidden_act": "gelu",
            
            # Other necessary parameters for pretraining
            "mask_feature_prob": 0.0,
            "mask_feature_length": 5,
            "ctc_loss_reduction": "sum",
            "ctc_zero_infinity": False,
            
            # Initialize with random weights
            "initializer_range": 0.02,
        }
    })
    
    # Check command line if no path provided
    if config_path is None:
        config_path = OmegaConf.from_cli().get("config")
    
    # Load and merge YAML config
    if config_path and os.path.exists(config_path):
        yaml_config = OmegaConf.load(config_path)
        config = OmegaConf.merge(default_config, yaml_config)
        print(f"Loaded pretraining config: {config_path}")
    else:
        config = default_config
        print("Using default pretraining config")
    
    # Validation (equivalent to argparse required fields and assertions)    
    if not config.dataset_config_names:
        raise ValueError("dataset_config_names is required")
    
    if not config.dataset_split_names:
        raise ValueError("dataset_split_names is required")
    
    # Create output directory if specified
    if config.output_dir is not None:
        os.makedirs(config.output_dir, exist_ok=True)
    
    return config

def config_to_namespace(config: DictConfig) -> Namespace:
    """Convert OmegaConf config to argparse Namespace for compatibility."""
    return Namespace(**OmegaConf.to_container(config, resolve=True))

def setup_pretraining_config(config_path: Optional[str] = None):
    """Load config and inject both config and args into calling module's globals."""
    config = load_pretraining_config(config_path)
    args = config_to_namespace(config)
    
    # Inject both config and args into calling module's globals
    caller_globals = sys._getframe(1).f_globals
    caller_globals.update(config)
    caller_globals['config'] = config
    caller_globals['args'] = args
    
    return config, args

In [3]:
setup_pretraining_config()
EXPERIMENT_IDENTIFIER = f"{args.experiment_name}-{args.dataset_name}"
if not args.output_dir:
    args.output_dir = os.path.join(args.base_path,"runs",EXPERIMENT_IDENTIFIER)

args.per_device_eval_batch_size = args.batch_size
args.per_device_train_batch_size = args.batch_size

# Import hyperparameter tuning libraries only if needed
if args.tune_hyperparams:
    try:
        import optuna
        print("Optuna imported successfully for hyperparameter tuning")
    except ImportError:
        print("Warning: Optuna not installed. Install with: pip install optuna")
        args.tune_hyperparams = False

Using default pretraining config


In [4]:
@dataclass
class DataCollatorForWav2Vec2Pretraining:
    """
    Data collator that will dynamically pad the inputs received and prepare masked indices
    for self-supervised pretraining.

    Args:
        model (:class:`~transformers.Wav2Vec2ForPreTraining`):
            The Wav2Vec2 model used for pretraining. The data collator needs to have access
            to config and ``_get_feat_extract_output_lengths`` function for correct padding.
        feature_extractor (:class:`~transformers.Wav2Vec2FeatureExtractor`):
            The processor used for processing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
        mask_time_prob (:obj:`float`, `optional`, defaults to :obj:`0.65`):
            Percentage (between 0 and 1) of all feature vectors along the time axis which will be masked for the contrastive task.
            Note that overlap between masked sequences may decrease the actual percentage of masked vectors.
            The default value is taken from the original wav2vec 2.0 article (https://arxiv.org/abs/2006.11477),
            and results in about 49 percent of each sequence being masked on average.
        mask_time_length (:obj:`int`, `optional`, defaults to :obj:`10`):
            Length of each vector mask span to mask along the time axis in the contrastive task. The default value
            originates from the original wav2vec 2.0 article and corresponds to the ``M`` variable mentioned there.
    """

    model: Wav2Vec2ForPreTraining
    feature_extractor: Wav2Vec2FeatureExtractor
    padding: Union[bool, str] = "longest"
    pad_to_multiple_of: Optional[int] = None
    mask_time_prob: Optional[float] = 0.65
    mask_time_length: Optional[int] = 10

    def __call__(self, features: list[dict[str, Union[list[int], torch.Tensor]]]) -> dict[str, torch.Tensor]:
        # reformat list to dict and set to pytorch format
        batch = self.feature_extractor.pad(
            features,
            padding=self.padding,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        device = batch["input_values"].device
        batch_size = batch["input_values"].shape[0]

        mask_indices_seq_length = self.model._get_feat_extract_output_lengths(batch["input_values"].shape[-1])
        # make sure masked sequence length is a Python scalar
        mask_indices_seq_length = int(mask_indices_seq_length)

        # make sure that no loss is computed on padded inputs
        if batch.get("attention_mask") is not None:
            # compute real output lengths according to convolution formula
            batch["sub_attention_mask"] = self.model._get_feature_vector_attention_mask(
                mask_indices_seq_length, batch["attention_mask"]
            )

        features_shape = (batch_size, mask_indices_seq_length)

        # sample randomly masked indices
        mask_time_indices = _compute_mask_indices(
            features_shape,
            self.mask_time_prob,
            self.mask_time_length,
            attention_mask=batch.get("sub_attention_mask"),
        )

        # sample negative indices
        sampled_negative_indices = _sample_negative_indices(
            features_shape,
            self.model.config.num_negatives,
            mask_time_indices=mask_time_indices,
        )
        batch["mask_time_indices"] = torch.tensor(mask_time_indices, dtype=torch.long, device=device)
        batch["sampled_negative_indices"] = torch.tensor(sampled_negative_indices, dtype=torch.long, device=device)

        return batch


def multiply_grads(params, c):
    """Multiplies grads by a constant *c*."""
    for p in params:
        if p.grad is not None:
            if torch.is_tensor(c):
                c = c.to(p.grad.device)
            p.grad.data.mul_(c)


def get_grad_norm(params, scale=1):
    """Compute grad norm given a gradient scale."""
    total_norm = 0.0
    for p in params:
        if p.grad is not None:
            param_norm = (p.grad.detach().data / scale).norm(2)
            total_norm += param_norm.item() ** 2
    total_norm = total_norm**0.5
    return total_norm

# Setup Functions

In [5]:
def setup_datasets(args):
    """Setup and process datasets."""
    # 1. Download and create train, validation dataset
    raw_datasets = DatasetDict()
    for dataset_config_name, split_name in zip(args.dataset_config_names, args.dataset_split_names):
        # load dataset
        path_data = os.path.join(args.base_path, "datasets", args.dataset_name, f"{split_name}.json")
        data = load_dataset("json", data_files=path_data, field="batches")
        raw_datasets[split_name] = data["train"]

    # Apply length filters
    max_length = args.max_trace_length
    min_length = args.min_trace_length

    if min_length > 0:
        before_count = len(raw_datasets["train"])
        raw_datasets = raw_datasets.filter(
            lambda x: x["trace_length"] > min_length,
            num_proc=args.preprocessing_num_workers,
        )
        after_count = len(raw_datasets["train"])
        print(f"Min length filter: {before_count} -> {after_count} (removed {before_count - after_count})")

    if max_length is not None:
        before_count = len(raw_datasets["train"])
        raw_datasets = raw_datasets.filter(
            lambda x: x["trace_length"] <= max_length,
            num_proc=args.preprocessing_num_workers,
        )
        after_count = len(raw_datasets["train"])
        print(f"Max length filter: {before_count} -> {after_count} (removed {before_count - after_count})")

    return raw_datasets


def setup_feature_extractor(args):
    """Setup feature extractor."""
    feature_extractor = Wav2Vec2FeatureExtractor(
        feature_size=1, 
        sampling_rate=args.sampling_rate, 
        padding_value=0.0, 
        do_normalize=args.normalize_features, 
        return_attention_mask=False
    )

    # # only normalized-inputs-training is supported
    # if not feature_extractor.do_normalize:
    #     raise ValueError(
    #         "Training is only supported for normalized inputs. Make sure ``feature_extractor.do_normalize == True``"
    #     )
    
    return feature_extractor


def expand_dataset_entries(dataset, args, feature_extractor):
    """Expand each matrix entry into N separate dataset entries"""
    expanded_data = {
        "input_values": [],
        "input_length": []
    }

    max_length = args.max_trace_length

    for example in dataset:
        # Load numpy trace data
        traces = np.load(example[args.data_column_name])

        # Handle both single traces and matrices
        if traces.ndim == 1:
            # Single trace - keep as one entry
            traces = [traces]
        else:
            # Matrix - each row becomes a separate entry
            traces = [traces[i] for i in range(traces.shape[0])]

        # Create separate dataset entries for each trace
        for i, trace in enumerate(traces):
            if i >= args.max_trace_expansion:
                break
            inputs = feature_extractor(
                trace, 
                sampling_rate=args.sampling_rate,
                max_length=max_length,
                truncation=True
            )

            expanded_data["input_values"].append(inputs.input_values[0])
            expanded_data["input_length"].append(len(inputs.input_values[0]))

    return expanded_data


def setup_vectorized_datasets(args, raw_datasets, feature_extractor, accelerator):
    """Setup vectorized datasets by expanding matrix files."""
    # load trace files
    with accelerator.main_process_first():
        print("Expanding matrix files into individual traces...")

        # Process train and validation separately
        train_expanded = expand_dataset_entries(raw_datasets["train"], args, feature_extractor)
        val_expanded = expand_dataset_entries(raw_datasets["val"], args, feature_extractor)

        print(f"Train: {len(raw_datasets['train'])} -> {len(train_expanded['input_values'])} examples")
        print(f"Validation: {len(raw_datasets['val'])} -> {len(val_expanded['input_values'])} examples")

        # Create new datasets from the expanded data
        vectorized_datasets = DatasetDict({
            "train": datasets.Dataset.from_dict(train_expanded),
            "val": datasets.Dataset.from_dict(val_expanded)
        })

        # Remove input_length column
        vectorized_datasets = vectorized_datasets.remove_columns("input_length")

    return vectorized_datasets

In [6]:
def setup_model(args):
    """Setup Wav2Vec2 model with configuration."""
    config = Wav2Vec2Config(
        # Core architecture parameters
        hidden_size=args.model["hidden_size"],
        num_hidden_layers=args.model["num_hidden_layers"],
        num_attention_heads=args.model["num_attention_heads"],

        # Feature extraction parameters
        conv_dim=args.model["conv_dim"],
        conv_stride=args.model["conv_stride"],
        conv_kernel=args.model["conv_kernel"],

        # Normalization settings
        feat_extract_norm=args.model["feat_extract_norm"],
        layer_norm_eps=args.model["layer_norm_eps"],
        do_stable_layer_norm=args.model["do_stable_layer_norm"],

        # Masking parameters for pretraining
        mask_time_prob=args.mask_time_prob,
        mask_time_length=args.mask_time_length,

        # Quantization/codebook parameters
        vocab_size=args.model["vocab_size"],
        num_codevectors_per_group=args.model["num_codevectors_per_group"],
        num_codevector_groups=args.model["num_codevector_groups"],
        num_negatives=args.model["num_negatives"],

        # Regularization
        hidden_dropout=args.model["hidden_dropout"],
        attention_dropout=args.model["attention_dropout"],
        activation_dropout=args.model["activation_dropout"],
        feat_proj_dropout=args.model["feat_proj_dropout"],
        layerdrop=args.model["layerdrop"],
        diversity_loss_weight=args.model["diversity_loss_weight"],

        # Activation function
        hidden_act=args.model["hidden_act"],

        # Other necessary parameters for pretraining
        mask_feature_prob=args.model["mask_feature_prob"], # FIXME: Unused?
        mask_feature_length=args.model["mask_feature_length"], # FIXME: Unused?
        ctc_loss_reduction=args.model["ctc_loss_reduction"],
        ctc_zero_infinity=args.model["ctc_zero_infinity"],

        # Initialize with random weights
        initializer_range=args.model["initializer_range"],
    )

    # pretraining is only supported for "newer" stable layer norm architecture
    if not config.do_stable_layer_norm or config.feat_extract_norm != "layer":
        raise ValueError(
            "PreTraining is only supported for ``config.do_stable_layer_norm=True`` and"
            " ``config.feat_extract_norm='layer'"
        )

    # initialize random model
    model = Wav2Vec2ForPreTraining(config)

    # Activate gradient checkpointing if needed
    if args.gradient_checkpointing:
        model.gradient_checkpointing_enable()

    return model

In [7]:
def setup_data_collator(args, model, feature_extractor):
    """Setup data collator for pretraining."""
    mask_time_prob = model.config.mask_time_prob if args.mask_time_prob is None else args.mask_time_prob
    mask_time_length = model.config.mask_time_length if args.mask_time_length is None else args.mask_time_length

    data_collator = DataCollatorForWav2Vec2Pretraining(
        model=model,
        feature_extractor=feature_extractor,
        pad_to_multiple_of=args.pad_to_multiple_of,
        mask_time_prob=mask_time_prob,
        mask_time_length=mask_time_length,
    )
    
    return data_collator


def setup_dataloaders(args, vectorized_datasets, data_collator):
    """Setup train and validation dataloaders."""

    print("Setting up dataloaders...")

    if args.group_by_length:
        # Add groupings to prevent excessive padding and wasted compute
        sampler = LengthGroupedSampler(
            batch_size=args.batch_size,
            dataset=vectorized_datasets["train"],
            lengths=[len(item["input_values"]) for item in vectorized_datasets["train"]],
        )

        train_dataloader = DataLoader(
            vectorized_datasets["train"],
            sampler=sampler,
            collate_fn=data_collator,
            batch_size=args.batch_size,
        )
    else:
        train_dataloader = DataLoader(
            vectorized_datasets["train"],
            shuffle=True,
            collate_fn=data_collator,
            batch_size=args.batch_size,
        )
    
    eval_dataloader = DataLoader(
        vectorized_datasets["val"], 
        collate_fn=data_collator, 
        batch_size=args.batch_size,
    )

    print(f"Train dataloader size: {len(train_dataloader)} batches")

    return train_dataloader, eval_dataloader


def setup_optimizer_and_scheduler(args, model, train_dataloader):
    """Setup optimizer and learning rate scheduler."""
    # Optimizer
    optimizer = torch.optim.AdamW(
        list(model.parameters()),
        lr=args.learning_rate,
        betas=[args.adam_beta1, args.adam_beta2],
        eps=args.adam_epsilon,
    )

    # Scheduler and math around the number of training steps.
    num_update_steps_per_epoch = math.ceil(len(train_dataloader) / args.gradient_accumulation_steps)

    if args.max_train_steps is None:
        args.max_train_steps = args.num_train_epochs * num_update_steps_per_epoch

    lr_scheduler = get_scheduler(
        name=args.lr_scheduler_type,
        optimizer=optimizer,
        num_warmup_steps=args.num_warmup_steps,
        num_training_steps=args.max_train_steps,
    )

    # Afterwards we recalculate our number of training epochs
    args.num_train_epochs = math.ceil(args.max_train_steps / num_update_steps_per_epoch)

    return optimizer, lr_scheduler

# Training Loop

In [8]:
def update_args_with_optuna_params(args, trial):
    """Update args with Optuna hyperparameter suggestions."""

    # args.batch_size = trial.suggest_categorical("batch_size", [16, 32, 48])
    # args.normalize_features = trial.suggest_categorical("normalize_features", [True, False])
    args.learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)
    args.mask_time_prob = trial.suggest_float("mask_time_prob", 0.05, 0.4, log=True)
    args.mask_time_length = trial.suggest_int("mask_time_length", 1, 8)
    
    # Model architecture parameters
    # args.model["num_attention_heads"] = trial.suggest_categorical("num_attention_heads", [2, 4, 8])
    args.model["num_hidden_layers"] = trial.suggest_categorical("num_hidden_layers", [6, 12])
    args.model["num_codevectors_per_group"] = trial.suggest_categorical("num_codevectors_per_group", [4, 6, 8])
    args.model["num_codevector_groups"] = trial.suggest_categorical("num_codevector_groups", [1, 2])
    args.model["diversity_loss_weight"] = trial.suggest_float("diversity_loss_weight", 0.5, 2.0)
    
    # Conv stride configuration
    conv_stride_options = [[4, 2, 2], [2, 2, 1], [2, 1, 1]]
    args.model["conv_stride"] = trial.suggest_categorical("conv_stride", conv_stride_options)
    
    return args

In [9]:
def train_model(args, accelerator, vectorized_datasets):
    total_batch_size = args.batch_size * accelerator.num_processes * args.gradient_accumulation_steps

    logger.info("***** Running training *****")
    logger.info(f"  Num examples = {len(vectorized_datasets['train'])}")
    logger.info(f"  Num Epochs = {args.num_train_epochs}")
    logger.info(f"  Instantaneous batch size per device = {args.batch_size}")
    logger.info(f"  Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
    logger.info(f"  Gradient Accumulation steps = {args.gradient_accumulation_steps}")
    logger.info(f"  Total optimization steps = {args.max_train_steps}")

    # Setup all components
    feature_extractor = setup_feature_extractor(args)
    model = setup_model(args)
    data_collator = setup_data_collator(args, model, feature_extractor)
    train_dataloader, eval_dataloader = setup_dataloaders(args, vectorized_datasets, data_collator)
    optimizer, lr_scheduler = setup_optimizer_and_scheduler(args, model, train_dataloader)
   
    model, optimizer, train_dataloader, eval_dataloader, lr_scheduler = accelerator.prepare(
        model, optimizer, train_dataloader, eval_dataloader, lr_scheduler
    )

    # set up weights and biases if available
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")

    if is_wandb_available():
        if wandb.run is not None:
            wandb.finish()

        wandb.init(
            project=args.output_dir.split("/")[-1], 
            dir=args.output_dir,
            name=timestamp,
            )
    
    # Set up logger
    logger.info(accelerator.state, main_process_only=False)
    if accelerator.is_local_main_process:
        datasets.utils.logging.set_verbosity_warning()
        transformers.utils.logging.set_verbosity_info()
    else:
        datasets.utils.logging.set_verbosity_error()
        transformers.utils.logging.set_verbosity_error()

    completed_steps = 0
    starting_epoch = 0
    # Only show the progress bar once on each machine.
    progress_bar = tqdm(range(args.max_train_steps), disable=not accelerator.is_local_main_process)
    
    for epoch in range(starting_epoch, args.num_train_epochs):
        model.train()
        for step, batch in enumerate(train_dataloader):
            # compute num of losses
            num_losses = batch["mask_time_indices"].sum()
            sub_attention_mask = batch.pop("sub_attention_mask", None)
            sub_attention_mask = (
                sub_attention_mask if sub_attention_mask is not None else torch.ones_like(batch["mask_time_indices"])
            )
            percent_masked = num_losses / sub_attention_mask.sum()

            # forward
            outputs = model(**batch)

            # divide loss by gradient accumulation steps since gradients
            # are accumulated for multiple backward passes in PyTorch
            loss = outputs.loss / args.gradient_accumulation_steps
            accelerator.backward(loss)

            # make sure that `num_losses` is summed for distributed training
            # and average gradients over losses of all devices
            if accelerator.state.num_processes > 1:
                num_losses = accelerator.gather_for_metrics(num_losses).sum()
                gradient_multiplier = accelerator.state.num_processes / num_losses
                multiply_grads(model.module.parameters(), gradient_multiplier)
            else:
                multiply_grads(model.parameters(), 1 / num_losses)

            # update step
            if (step + 1) % args.gradient_accumulation_steps == 0 or step == len(train_dataloader) - 1:
                # compute grad norm for monitoring
                scale = (
                    accelerator.scaler._scale.item()
                    if hasattr(accelerator, "scaler") and accelerator.scaler is not None
                    else 1
                )
                if accelerator.state.num_processes > 1:
                    grad_norm = get_grad_norm(model.module.parameters(), scale)
                else:
                    grad_norm = get_grad_norm(model.parameters(), scale)

                # update parameters
                optimizer.step()
                optimizer.zero_grad()

                if not accelerator.optimizer_step_was_skipped:
                    lr_scheduler.step()
                elif accelerator.is_local_main_process:
                    progress_bar.write(
                        f"Gradients have overflown - skipping update step... Updating gradient scale to {scale}..."
                    )

                # update gumbel temperature
                gumbel_temperature = max(
                    args.max_gumbel_temperature * args.gumbel_temperature_decay**completed_steps,
                    args.min_gumbel_temperature,
                )
                if hasattr(model, "module"):
                    model.module.set_gumbel_temperature(gumbel_temperature)
                else:
                    model.set_gumbel_temperature(gumbel_temperature)

                progress_bar.update(1)
                completed_steps += 1

            # 6. Log all results
            if (step + 1) % (args.gradient_accumulation_steps * args.logging_steps) == 0:
                loss.detach()
                outputs.contrastive_loss.detach()
                outputs.diversity_loss.detach()

                if accelerator.state.num_processes > 1:
                    loss = accelerator.gather_for_metrics(loss).sum()
                    outputs.contrastive_loss = accelerator.gather_for_metrics(outputs.contrastive_loss).sum()
                    outputs.diversity_loss = accelerator.gather_for_metrics(outputs.diversity_loss).sum()
                    percent_masked = accelerator.gather_for_metrics(percent_masked).sum()

                train_logs = {
                    "loss": (loss * args.gradient_accumulation_steps) / num_losses,
                    "constrast_loss": outputs.contrastive_loss / num_losses,
                    "div_loss": outputs.diversity_loss / num_losses,
                    "%_mask_idx": percent_masked / accelerator.num_processes,
                    "perplexity": outputs.codevector_perplexity,
                    "lr": torch.tensor(optimizer.param_groups[0]["lr"]),
                    "temp": torch.tensor(gumbel_temperature),
                    "grad_norm": torch.tensor(grad_norm),
                }
                log_str = ""
                for k, v in train_logs.items():
                    log_str += f"| {k}: {v.item():.3e}"

                if accelerator.is_local_main_process:
                    progress_bar.write(log_str)
                    if is_wandb_available():
                        wandb.log(train_logs)

            # if completed steps > `args.max_train_steps` stop
            if completed_steps >= args.max_train_steps:
                break
        
        # 7. Validate!
        model.eval()

        # init logs
        val_logs = {
            "val_loss": 0,
            "val_contrastive_loss": 0,
            "val_diversity_loss": 0,
            "val_num_losses": 0,
        }
        for step, batch in enumerate(eval_dataloader):
            with torch.no_grad():
                batch.pop("sub_attention_mask", None)
                outputs = model(**batch)

            val_logs["val_loss"] += outputs.loss
            val_logs["val_contrastive_loss"] += outputs.contrastive_loss
            val_logs["val_diversity_loss"] += outputs.diversity_loss
            val_logs["val_num_losses"] += batch["mask_time_indices"].sum()

        # sum over devices in multi-processing
        if accelerator.num_processes > 1:
            val_logs = {k: accelerator.gather_for_metrics(v).sum() for k, v in val_logs.items()}

        val_logs = {k: v / val_logs["val_num_losses"] for k, v in val_logs.items()}
        val_loss_final = val_logs["val_loss"].item()

        log_str = ""
        for k, v in val_logs.items():
            log_str += f"| {k}: {v.item():.3e}"

        if accelerator.is_local_main_process:
            progress_bar.write(log_str)
            if is_wandb_available():
                wandb.log(val_logs)

        if (args.output_dir is not None) and (not args.tune_hyperparams):
            accelerator.wait_for_everyone()
            unwrapped_model = accelerator.unwrap_model(model)
            unwrapped_model.save_pretrained(
                args.output_dir, is_main_process=accelerator.is_main_process, save_function=accelerator.save
            )

    return {"val_loss": val_loss_final}

# Hyperparam Loop

In [10]:
def hyperparameter_tuning(args, accelerator, vectorized_datasets):
    """Perform hyperparameter tuning using Optuna"""
    print("Starting hyperparameter tuning...")
    
    def objective(trial):
        # Clear CUDA cache
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        
        # Create a copy of args to avoid modifying the original
        trial_args = Namespace(**vars(args))

        # Update search space directly in the args variable
        trial_args = update_args_with_optuna_params(trial_args, trial)

        # Train with trial parameters
        try:
            result = train_model(trial_args, accelerator, vectorized_datasets)
            val_loss = result["val_loss"]
            print(f"Trial {trial.number} completed with val_loss: {val_loss:.4f}")
            return val_loss
        except Exception as e:
            print(f"Trial {trial.number} failed with error: {e}")
            return float('inf')
    
    # Create study
    db_filename = "optuna.db"
    db_path = os.path.join(args.output_dir, db_filename)
    print(args.output_dir)
    os.makedirs(args.output_dir, exist_ok=True)
    
    study = optuna.create_study(
        direction="minimize",
        study_name="wav2vec2_tuning_loss",
        storage=f"sqlite:///{db_path}",
        load_if_exists=True
    )
    
    # Optimize
    study.optimize(objective, n_trials=200, timeout=3600*15)  # 15 hours max
    
    print("Hyperparameter tuning completed!")
    print(f"Best val_loss: {study.best_value:.4f}")
    print("Best parameters:")
    for key, value in study.best_params.items():
        print(f"  {key}: {value}")
    
    return study.best_params

# Main

In [11]:
# Sending telemetry. Tracking the example usage helps us better allocate resources to maintain them. The
# information sent is the one passed as arguments along with your Python/PyTorch versions.
send_example_telemetry("run_wav2vec2_pretraining_no_trainer", args)

# Initialize the accelerator. We will let the accelerator handle device placement for us in this example.
accelerator = Accelerator()

# If passed along, set the training seed now.
if args.seed is not None:
    set_seed(args.seed)

accelerator.wait_for_everyone()

raw_datasets = setup_datasets(args)
feature_extractor = setup_feature_extractor(args)
vectorized_datasets = setup_vectorized_datasets(args, raw_datasets, feature_extractor, accelerator)

if args.tune_hyperparams:
    # Perform hyperparameter tuning
    best_params = hyperparameter_tuning(args, accelerator, vectorized_datasets)
    print("Optuna tuning completed.")
else:
    # Train the model (either with original params or best params from tuning)
    result = train_model(args, accelerator, vectorized_datasets)
    print(f"Training completed with final val_loss: {result['val_loss']:.4f}")

FileNotFoundError: Unable to find '/workspaces/asr4scbd/./datasets/pre_rvi_mdb/train.json'